In [1]:
import pandas as pd


In [2]:
dataframe = pd.read_csv('./voice.csv')
#这个是为了做显示输出所用
pd.set_option('display.max_columns', 500)
print(dataframe.shape)

(3168, 21)


In [3]:
#shape[0]是行数，【1】是列数
print('{}'.format(dataframe.shape[0]))

3168


In [4]:
# -1是要和不要最后一行，voice.csv中的数据最后一列是label. X肯定是不要，y是要的。
X = dataframe.iloc[:, :-1]
y = dataframe.iloc[:, -1]

In [5]:
#做 lable  encoder 
from sklearn.preprocessing import LabelEncoder, StandardScaler
gender_encoder = LabelEncoder()
y = gender_encoder.fit_transform(y)

In [6]:
#做z-score，正态
from sklearn import preprocessing
ss = preprocessing.StandardScaler()
X = ss.fit_transform(X)


In [7]:
# 分割数据，将25%的数据作为测试集，其余作为训练集
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state=33)

In [8]:
import xgboost as xgbo
param = {'boosting_type' : 'gbdt',
                         'objective' : 'binary:logistic', #任务目标
                         'eval_metric' : 'auc', #评估指标
                         'eta' : 0.01, #学习率
                         'max_depth' : 20, #树最大深度
                         'colsample_bytree':0.8, #设置在每次迭代中使用特征的比例
                         'subsample' : 0.8, #样本采样比例
                         'subsample_freq' : 10, #bagging的次数
                         'alpha': 0.6, #L1正则
                         'lambda': 0, #L2正则
        }
train_data = xgbo.DMatrix(train_x, train_y)
test_data = xgbo.DMatrix(test_x, test_y) 
print(test_y)
model = xgbo.train(param,  
                   train_data, evals= [(train_data, 'train'), (test_data, 'valid')],
                   num_boost_round=3000,
                   early_stopping_rounds=25, verbose_eval=25
                  )
y_pred = model.predict(test_data)
#必须作如下处理，否则在accuracy_score时，会报错。
y_pred = [1 if x>=0.5 else 0 for x in y_pred]
from sklearn.metrics import accuracy_score
print('XGBoost :', accuracy_score(test_y, y_pred ))

[1 1 0 1 1 0 0 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1 1 1 0
 0 0 1 1 1 0 1 0 0 1 0 0 0 1 1 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 1 0 0 0 0 1 0
 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 1 1 1 1 0 0 1 1 0 0 1 0 1 0
 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 1 0 1 0 1 1 1 1 1 0 0 1 0 1 1 1
 1 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 0
 0 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1
 1 0 0 1 1 1 0 1 0 0 0 0 1 0 1 1 0 1 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0
 0 1 0 1 1 1 0 0 1 1 0 1 1 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1
 0 1 0 1 1 0 1 0 0 1 1 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1
 0 0 1 1 0 0 0 0 0 1 1 1 1 0 1 1 1 1 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 1 0 1
 0 1 1 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 1 1 0 1 0 1 0 1
 0 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 1 0 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1
 1 0 1 0 0 0 0 0 1 0 1 1 